In [4]:
from main import *
import sqlalchemy as sa

In [6]:
tweet_obj, user_obj = get_files_and_insert()

mentoo_search_2018-01-01_2019-01-01.jsonl
mentoo_search_2019-01-01_2020-01-01.jsonl
mentoo_search_2020-01-01_2021-01-01.jsonl
mentoo_search_2021-01-01_2022-01-01.jsonl
mentoo_search_2022-01-01_2022-06-01.jsonl


In [6]:
# tweet_obj.tables[cur_table].drop_duplicates(subset=['id'], inplace=True)

In [8]:
q = """
SELECT * FROM information_schema.columns
WHERE table_name = 'hashtag_tweet_mapping'
"""

In [7]:
tweet_obj.tables["hashtag_tweet_mapping"]

,tweet_id,tag,start,end
0,1531787586280124417,AmberHeard,0,11
0,1531787586280124417,AmberTurd,12,22
0,1531787586280124417,MePoo,23,29
0,1531787586280124417,AmberHeardIsALiar,32,50
0,1531787586280124417,AmberHeardIsAnAbuser,52,73
...,...,...,...,...
7781,1531479136170127360,JusticeForJohnnyDepp,193,214
7781,1531479136170127360,JusticeForJohnny,215,232
7781,1531479136170127360,IStandWithJohnnyDepp,233,254
7781,1531479136170127360,MenToo,255,262


In [9]:
table_schema = pd.read_sql(q, engine)

NameError: name 'engine' is not defined

In [13]:
# set(tweet_obj.tables['replied_to_tweet_mapping'].columns)
set(table_schema["column_name"].unique()) == set(
    tweet_obj.tables["hashtag_tweet_mapping"].columns
)

True

In [12]:
q = """
SELECT count(*) FROM
author_data
"""

In [10]:
q = """
select count(distinct(id)) from author_data
"""

In [13]:
pd.read_sql(q, engine)

,count
0,2092


In [20]:
q = """
SELECT qt.referenced_tweet_id,td.text,td.id
FROM quoted_tweet_mapping  qt
join tweet_data td
    on qt.referenced_tweet_id = td.id
"""

In [17]:
tweet_obj.tables["tweet_data"].id.count()

6761

In [21]:
quote_data = pd.read_sql(q, engine)

In [26]:
q = """
SELECT *
from mention_tweet_mapping
limit 10
"""

metioned_data = pd.read_sql(q, engine)

In [27]:
metioned_data

,tweet_id,mention_user_id,username,start,end
0,1079874246128660483,2308699184,MentalStigmas,3,17
1,1079874246128660483,5988062,TheEconomist,109,122
2,1079857715684564994,4787013172,nitin33K,3,12
3,1079857715684564994,4573405572,MumbaiPolice,14,27
4,1079857715684564994,3041294294,IPS_Association,106,122
5,1079857715684564994,2686770528,_DigitalIndia,123,137
6,1079848933504045056,4787013172,nitin33K,3,12
7,1079848933504045056,4573405572,MumbaiPolice,14,27
8,1079848933504045056,3041294294,IPS_Association,106,122
9,1079848933504045056,2686770528,_DigitalIndia,123,137
